Note that the labelling of these sections is based on the version of the paper completed on 15/08/22. This cell should be updated in the event of a change to the labelling in the paper.

Currently running on Sagemath version 9.7.beta8, Release Date: 2022-08-07.
Using Python 3.10.5.

This notebook is intended to be ran sequentially from the beginning. Changing the order in which cells are ran may lead to errors. 

## Proposition 2.13

In [1]:
# We will calculate the automorphism group of the curve numerically to check that it is indeed S5. 
# Let's define the curve
from sage.schemes.riemann_surfaces.riemann_surface import RiemannSurface
R.<x, y> = QQ[]
f = x*(y^5+1) + (x*y)^2 - x^4*y - 2*y^3
integration_method = 'rigorous'
prec = 60
S = RiemannSurface(f, prec=prec, integration_method=integration_method)
G = S.symplectic_automorphism_group()
# Torelli's theorem tell's us that the automorphism group of the Jacobian of a non-hyperelliptic
# curve X is C2 x Aut(X), so we get the result from the following.  
print(G.structure_description())

C2 x S5


## Theorem 2.14

In [2]:
# We first verify that the matrices written down do generate a group isomorphic to A5
K.<z> = CyclotomicField(5)
# We want to write down sqrt(5) in terms of the 5th root zeta
rs = (polygen(K)^2-5).roots(multiplicities=False)
# Let's check which one is sqrt(5), and which is -sqrt(5)
s5 = CC(sqrt(5))
remainders = [(K.embeddings(CC)[0](r)-s5).abs() for r in rs]
# Based on the above, we can write sqrt(5) in terms of z(eta)
s5zs = rs[remainders.index(min(remainders))]

# Write down the matrices
S = Matrix([[1,0,0],[0,z^(1),0],[0,0,z^(-1)]])
R = Matrix([[1,2,2],[1,z^2+z^(-2),z^1+z^(-1)],[1,z^1+z^(-1),z^2+z^(-2)]])/s5zs

print(MatrixGroup([R, S]).structure_description()=='A5')

True


In [3]:
# We verify the claims about the pencil of curves
P.<X,Y,Z> = K[]
F = X*(Y^5+Z^5)  + (X*Y*Z)^2 - X^4*Y*Z - 2*(Y*Z)^3
I2, I6 = [(X/2)^r + sum([(X/2+Y*z^k+Z*z^(-k))^r/(5^(r/2)) for k in range(5)]) for r in [2, 6]]

f_lambda = lambda l: I6 - l*I2^3

print(f_lambda(13/100) == (3/25)*F)
print(Curve(f_lambda(17/180)).change_ring(QQ).genus() == 0)
print(not Curve(f_lambda(1/10)).change_ring(QQ).is_irreducible())

True
True
True


## Proposition 2.15

In [4]:
## We now demonstrate how we found the automorphism U
# We need to redefine S as we have overwritten it earlier (in the name of keeping notation clear)
R.<x, y> = QQ[]
f = x*(y^5+1) + (x*y)^2 - x^4*y - 2*y^3
integration_method = 'rigorous'
prec = 60
S = RiemannSurface(f, prec=prec, integration_method=integration_method)
g = S.genus
RG = S.symplectic_isomorphisms()
Tnum = S.tangent_representation_numerical(RG)
# Recall that using the knowledge that we expect our matrices to be defined over K, we numerically find this 
# representation
# We adapt the code of tangent_representation_algebraic
zc = S._CC(z)
indeps = [zc^m for m in range(z.multiplicative_order()-1)]

def lincomb(v):
    ls = [v]+indeps
    w = vector(K, pari.lindep(ls).sage()) 
    if w[0] != 0 :
        return K(list(-w[1:]/w[0]))
    
Talg = [matrix(K, g, g, [lincomb(a) for a in tnum.list()]) for tnum in Tnum]

# We now predefine some terms necessary for the calculation
L = PolynomialRing(K, names=['x','y'])
# Note this depends on the order in which the differentials are defined by Sage.
# To be robust one should either manually enter these in the order desired or use some method to 
# check the order. 
vB = vector([L(b) for b in S.cohomology_basis()])

# One should look at the cohomology basis numerators to observe that these ratios do indeed give us x', y'.  
def mat_to_subs(M):
    w = M*vB
    yp = w[3]/w[0]
    xp = (w[1]+yp*w[3])/w[2]
    return (xp, yp)

# We write all the automorphisms of the HC model as substitutions in terms of affine coordinates x, y. 
auts_as_subs = [mat_to_subs(talg) for talg in Talg]

# We expect to actually find this is twice as large, so we can trim it
half_len = len(auts_as_subs)/2
if all([auts_as_subs.index(auts_as_subs[-i])==half_len-i for i in range(1, half_len)]):
    print("removing redundancy")
    auts_as_subs = auts_as_subs[:half_len]   
    
# We will then extract the data of which of these automorphisms are non-linear, and find the simplest
non_lin = []
for i in range(120):
    xp, yp = auts_as_subs[i]
    xnum = xp.numerator()
    xden = xp.denominator()
    ynum = yp.numerator()
    yden = yp.denominator()
    nds = [xnum, xden, ynum, yden]
    if not all([ndi.degree()<=1 for ndi in nds]):
        non_lin.append(auts_as_subs[i])

if len(non_lin) != 60:
    raise ValueError
    
lengths = [len(aut[0]._repr_())+len(aut[1]._repr_()) for aut in non_lin]
nl = non_lin[lengths.index(min(lengths))]
print("x' = {}".format(nl[0].factor()))
print("y' = {}".format(nl[1].factor()), "\n")

removing redundancy
x' = (x^2 - y)^-1 * (y^3 - x)^-1 * (y^5 + x^3*y - 3*x*y^2 + 1)
y' = (-1) * (y^3 - x)^-1 * (x*y^2 - 1) 



## Action on $\mathbb{P}^1 \times \mathbb{P}^1$

In [5]:
# We will now demonstrate how to get the action of the automorphism group on P1xP1. 
# so as to not overlap notation we will now call the generator of the cycltomic field c.
K.<c> = CyclotomicField(5)
P.<u,v,z,w> = K[]
# We crete the vecor of the (modified) Sgre embedding
vec = vector([u*z, v*w, v*z, -u*w])

# The matrices A and B map from the coordinates [vi:] coming from the cohomology basis to 
# the coordinates [xi:] of the standard form. It is in these latter coordinate that we know 
# how to write down the transform (12) easily, so we carry that action over.
A = matrix([[ c^3,   -1, -c^2,    c], [   1, -c^3,   -c,  c^2], 
            [ c^2,   -c,   -1,  c^3], [   c, -c^2, -c^3,    1]])
B = matrix([[0, 0, 0, 1], [0, 0, -1, 0], [0, 1, 0, 0], [-1, 0, 0, 0]])
AB = A*B
ABi = AB.inverse()

# We transform the vector
x = AB*vec
t_12 = matrix([[0, 1, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
x_t = t_12*x
vec_t = ABi*x_t

# As the transform will be an element of PGL(2, C) on each P1 we can read off the 
# corresponding matrix which we print off. 
vou = vec_t[2]/vec_t[0]
woz = vec_t[1]/vec_t[2]

vou_m = matrix([list(reversed(vou.numerator().coefficients())), 
                list(reversed(vou.denominator().coefficients()))])
woz_m = matrix([list(reversed(woz.numerator().coefficients())), 
                list(reversed(woz.denominator().coefficients()))])


print(vou_m/vou_m.det())
print()
print(woz_m/woz_m.det())

[-c^3 - c^2         -1]
[        -1  c^3 + c^2]

[            -1 -c^3 - c^2 - 1]
[-c^3 - c^2 - 1              1]
